# IBM & DataStax Demo - Banking AI Agent

## Before start

What you will need:

- Install dependencies (check the README.md)
- An account on DataStax Astra (Part I)
- A API Key from Watsonx.AI or OpenAI

## Part I - Astra Setup

- Create an account on DataStax Astra.
- Create a Database
- Create a collection with Vectorize (NVIDIA model)
- Load the Astra collection with a document

## Part II - Langflow RAG

- Create a Langflow RAG flow and connect to Astra
- Activate the NVIDIA reranker
- Run the Flow through the Langflow API

## Part III - Agents and NoSQL

- Create a CQL table to store banking transactions
- Load sample data
- Create an Banking Agent Flow on Langflow.
- Connect the Astra DB Tools to the agent.
- Run the Flow through the Langflow API


## Installing dependencies

Check the [README.md](./README.md) to set up your enviroment before startinge the execution of this notebook.

## Creating the DataStax Astra Account

>INFO: On DataStax Astra, every user receives $25 in credits *EVERY MONTH* to run Astra DB, Astra Streaming and Langflow.

To start using the platform, access [astra.datastax.com](astra.datastax.com) and click on the "Sign Up" link to start the registration.

<img src="img/signup.png" alt="Sign Up Page" width="600"/>

After the Sign Up process, you should be able to login in to the Astra Dashboard

# Part I - Astra DB Setup

## Creating a Database

After logging in on Astra Dashboard, click on the "Create Database" button.

<img src="img/dashboard.png" alt="Start Database creation" width="600"/>

On the Create Database screen, fill

- Deployment Type: Serverless (Vector)
- Provider: AWS
- Region: us-east-2

<img src="img/create_db.png" alt="DB Creation" width="600"/>

> INFO: It is possible to create DBs on AWS, Azure and GCP. To check all available regions, check our [documentation](https://docs.datastax.com/en/astra-db-serverless/databases/regions.html)

After 2 or 3 minutes, your database will be ready to start using it.

<img src="img/db_created.png" alt="DB Created" width="600"/>

More info available [here](https://docs.datastax.com/en/astra-db-serverless/databases/create-database.html)


## Creating the Collection

> INFO: Astra DB can store __collections__ and __tables__
>
> [__Collections__](https://docs.datastax.com/en/astra-db-serverless/api-reference/collections.html) are used to store JSON documents with a semi structured data model. It is compatible with Mongo DB and [MongooseJS](https://docs.datastax.com/en/astra-db-serverless/integrations/data-api-with-mongoosejs.html). 
>
> [__Tables__](https://docs.datastax.com/en/astra-db-serverless/cql/develop-with-cql.html) are used to store transactional data that requires high throughput and low latencies. Due to its data model it is possible to achieve tens of thousands operations per second. 
>
> Both models are compatible with __Vector Search__


On the DB Dashboard, click on __"Data Explorer"__ and then click on __"Create Collection +"__

<img src="img/start_create_collection.png" alt="Start collection creation" width="600"/>

The screen for the collection creation should be filled with:

- Collection name: banking_knowledge_layer
- Vector-enabled collection: Activated
- Embedding generation method: NVIDIA (The embedding model and dimension will be filled automatically)
- Similarity Metric: Cosine (Dot Product and Euclidean are additional options)

<img src="img/create_collection.png" alt="Create collection" width="600"/>


> ### Vectorize
> 
> 
> When creating a collection, you can automate the embedding generation process using [__Vectorize__](https://docs.datastax.com/en/astra-db-serverless/databases/embedding-generation.html).
>
> In this example, we're using NVIDIA models provided by DataStax. However, you can also integrate with other model providers such as OpenAI, Azure OpenAI, Hugging Face, and more.
> 
> With this setup, all embedding generation—whether during data loading or querying—is seamlessly handled at the database layer.


## Loading data.

There are multiple ways to load data into Astra. For this part of demo, we'll use __LangChain__ to showcase the developer experience we offer for users who want full control over the entire process.

## Get the token and API Endpoint

Start by copying the "_env_sample" file to ".env" file. We will store the Astra token information on the env file.

On the Astra Dashboard, access the db "astra_ibm_demo", click on the "Overview" tab and find the "Database Details" on the right side of the screen.

<img src="img/token.png" alt="Token" width="600"/>

Copy the API Endpoint and save it to the ".env" file.

On the "Application Tokens" block, click on "Generate Token". It will open another block to generate the token. 

You can give some name to the token:

<img src="./img/generate_token.png" alt="Token" width="600"/>

The token will be created. Click on the "Copy" button and paste the value on your ".env" file.

Your ".env" file will be looking like this:

```bash
ASTRA_DB_APPLICATION_TOKEN="<your token>"
ASTRA_DB_API_ENDPOINT="<your db endpoint>"
IBM_WATSON_TOKEN=""
OPENAI_API_TOKEN=
```

Let's check if the environment variables are created:



In [4]:
#Make sure the environment variables are set
import os
from dotenv import load_dotenv
load_dotenv(override=True)

if os.getenv("ASTRA_DB_APPLICATION_TOKEN") is None:
    raise ValueError("Environment variable ASTRA_DB_APPLICATION_TOKEN not set")

if os.getenv("ASTRA_DB_APPLICATION_TOKEN")[:8] != "AstraCS:":
    raise ValueError("Environment variable ASTRA_DB_APPLICATION_TOKEN invalid format")

if ".apps.astra.datastax.com" not in os.getenv("ASTRA_DB_API_ENDPOINT"):
    raise ValueError("Environment variable ASTRA_DB_API_ENDPOINT invalid")

print(f'Astra Token: {os.getenv("ASTRA_DB_APPLICATION_TOKEN")[:10]}...{os.getenv("ASTRA_DB_APPLICATION_TOKEN")[-5:]}')
print(f'Astra Endpoint: {os.getenv("ASTRA_DB_API_ENDPOINT")}')
print("Good to go!")

Astra Token: AstraCS:bf...35ec6
Astra Endpoint: https://0df6bdb1-a6ca-4420-ba2e-f28550b3d178-us-east-2.apps.astra.datastax.com
Good to go!


### Data load

We will load public files from the Chase bank. We will use this documents as the source for questions answering and Agentic RAG.

The files are available on the ```docs``` folder

In [48]:
# Importing necessary libraries
from langchain_astradb import AstraDBVectorStore
from langchain_astradb.utils.astradb import HybridSearchMode
from astrapy.info import (
            CollectionLexicalOptions,
            CollectionRerankOptions,
            RerankServiceOptions,
            VectorServiceOptions,
        )
import pandas as pd


In [66]:
# Initializing the AstraDBVectorStore
vector_store = AstraDBVectorStore(
    collection_name="banking_knowledge_layer",
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    autodetect_collection=True,
    hybrid_search=HybridSearchMode.OFF
)

In [112]:
# Cleaning up the collection to start fresh
# WARNING: This will delete all documents in the collection
vector_store.clear() 

In [113]:
# Loading the documents from the specified directory
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the documents from the specified directory
loader = DirectoryLoader(
    "./docs",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=80,
)
documents = loader.load()
texts = text_splitter.split_documents(documents)
print(f"Loaded {len(texts)} chunks from the directory.")

# Add the documents to the vector store
vector_store.add_documents(texts)
print(f"Added {len(texts)} chunks to the vector store.")

Loaded 852 chunks from the directory.
Added 852 chunks to the vector store.


### Querying data.

With the knowledge base loaded, we can begin querying the vector store to retrieve passages relevant to a given question. This is the foundation of [RAG (Retrieval-Augmented Generation)](https://www.datastax.com/blog/what-is-rag-retrieval-augmented-generation).

Astra DB is capable of storing [millions of embeddings](https://www.datastax.com/press-release/wikimedia-deutschland-launches-ai-knowledge-project-in-collaboration-with-datastax-built-with-nvidia-ai). However, to achieve optimal performance and relevance, it’s a best practice to filter results using metadata whenever possible.

In the next steps, we’ll apply a metadata filter to target a specific document.

In the first query, we will run a query and return the top 3 most relevant chunks using only the vector search.

In [8]:
QUERY = "Can I refund a lost ticket?"
SOURCE_DOCUMENT = "docs/BGC10981_SapphireReserve_VisaInfinite.pdf"

def print_docs(docs):
    for doc in docs:
        print(f"Title: {doc.metadata['title']}")
        print(f"Subject: {doc.metadata['subject']}")
        print(f"Page Content: {doc.page_content}")
        print("-" * 80)

def astra_vector_search(query, source, k=3):
    # Perform a similarity search with the Hybrid search disabled
    vector_store = AstraDBVectorStore(
        collection_name="banking_knowledge_layer",
        api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
        token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
        autodetect_collection=True,
        hybrid_search=HybridSearchMode.OFF # Hybrid search disabled, so basic vector search is used
    )
    docs = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "filter": {"source": source},
            "k": k
        }
    ).invoke(query)
    
    return docs

# Create a DataFrame to display the docs in a table format
docs = astra_vector_search(QUERY,SOURCE_DOCUMENT,3)
print_docs(docs)

Title: Guide to Benefits
Subject: Guide
Page Content: repatriate the remains. All costs are Your responsibility.
•  Emergency Ticket Replacement helps You through Your carrier’s 
lost ticket reimbursement process and assists in the delivery of a 
replacement ticket to You, should You lose Your ticket. All costs are 
Your responsibility.
•  Lost Luggage Locator Service helps You through the Common 
Carrier’s claim procedures or can arrange shipment of replacement 
items if an airline or Common Carrier loses Your checked luggage.
--------------------------------------------------------------------------------
Title: Guide to Benefits
Subject: Guide
Page Content: 21
If you're outside of the US, call collect at 1-804-281-5772
Lost Luggage Reimbursement
The Lost Luggage benefit provides reimbursement for the difference 
between the actual cash value (replacement cost less depreciation) and 
any reimbursement provided by the Common Carrier for the costs You 
incur to repair or replace Your C

The results are ok, but the returned chunks don't directly answer the question.

Let’s enable reranking to improve the quality and relevance of the responses.

### Integrated Reranking

When developing a RAG (Retrieval-Augmented Generation) application, the results retrieved from Astra may sometimes require refinement to enhance accuracy.

Fortunately, [Astra has reranking built into its API](https://docs.datastax.com/en/astra-db-serverless/api-reference/document-methods/find-and-rerank.html), allowing us to retrieve reranked documents by default—no additional configuration needed.

> INFO: Astra DB Hybrid Search is enhanced with server-side reranking using the NVIDIA NeMo Retriever reranking microservices - built with NVIDIA NIM, part of the NVIDIA AI Enterprise software.
>
> Check this [blog post](https://www.datastax.com/blog/introducing-astra-db-hybrid-search) for more information

Let’s run the same query again and compare the results.

In [9]:
def astra_vector_search_and_rerank(query, source, k=3):
    # Lets recreate the vector store with hybrid search enabled
    vector_store = AstraDBVectorStore(
        collection_name="banking_knowledge_layer",
        api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
        token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
        autodetect_collection=True,
        hybrid_search=HybridSearchMode.ON # Hybrid search enabled
    )

    docs_reranked = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "filter": {"source": source},
            "k": k
        }
    ).invoke(query)
    return docs_reranked

docs_reranked = astra_vector_search_and_rerank(QUERY,SOURCE_DOCUMENT,3)
print_docs(docs_reranked)

/Users/samuel.matioli/work/astra-ibm-demo-2/venv/lib/python3.11/site-packages/langchain_astradb/vectorstores.py:2677: BetaFeatureWarning: Method 'Collection.find_and_rerank' is in beta and might undergo signature or behaviour changes in the future.
  hybrid_reranked_results = self.astra_env.collection.find_and_rerank(


Title: Guide to Benefits
Subject: Guide
Page Content: 41
If you're outside of the US, call collect at 1-804-281-5772
necessary medical treatment, to the airport, terminal or station of 
departure, and/or between the arrival airport, terminal or station 
and Your residence. This does not include transportation in vehicles 
operated by a medical facility or specifically designed to transport sick 
or injured individuals.
If You are forced to temporarily postpone a Trip due to a loss and a 
new departure date is set, We will reimburse for the prepaid unused
--------------------------------------------------------------------------------
Title: Guide to Benefits
Subject: Guide
Page Content: repatriate the remains. All costs are Your responsibility.
•  Emergency Ticket Replacement helps You through Your carrier’s 
lost ticket reimbursement process and assists in the delivery of a 
replacement ticket to You, should You lose Your ticket. All costs are 
Your responsibility.
•  Lost Luggage Loc

Lets put them side by side:

In [25]:
QUERY = "Is lugagge lost covered?"
SOURCE_DOCUMENT = "docs/BGC10981_SapphireReserve_VisaInfinite.pdf"

docs = astra_vector_search(QUERY,SOURCE_DOCUMENT,3)
docs_reranked = astra_vector_search_and_rerank(QUERY,SOURCE_DOCUMENT,3)

# Prepare data for the table
data = {
    "Docs": [doc.page_content for doc in docs],
    "Docs Reranked": [doc.page_content for doc in docs_reranked]
}

# Create a DataFrame
comparison_df = pd.DataFrame(data)

with pd.option_context('display.max_colwidth', None):
    display(comparison_df)
    
    

/Users/samuel.matioli/work/astra-ibm-demo-2/venv/lib/python3.11/site-packages/langchain_astradb/vectorstores.py:2677: BetaFeatureWarning: Method 'Collection.find_and_rerank' is in beta and might undergo signature or behaviour changes in the future.
  hybrid_reranked_results = self.astra_env.collection.find_and_rerank(


,Docs,Docs Reranked
0,"three (3) years after the time for giving proof of loss. Further, no \nlegal action may be brought against the Provider unless all the terms \nof the Guide to Benefits have been complied with fully. \n• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Federal Insurance Company. Coverage \nmay not be available in all states or certain terms may be different \nwhere required by state law.",sequence to the loss. War does not include terrorism.\nWhat’s Not Covered?\nThe Lost Luggage benefit does not apply to loss or theft of \nthe following: \n• Items that have been removed from Carry–On Baggage by You and \ninadvertently left behind on the Common Carrier or Cruise Line\n• Documents or valuable papers\n• Money\n• Securities\n• Tickets\n• Checks\n• Travelers’ checks\n• Furs\nHow Do You File a Claim?\nFollow these steps to file Your claim:
1,• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Indemnity Insurance Company of \nNorth America (“Provider”). Coverage may not be available in all \nstates or certain terms may be different where required by state law. \n• No legal action for a claim may be brought against the Provider \nuntil sixty (60) days after the Provider receives Proof of Loss.,"her Account for a Trip, each Covered Person is eligible to receive \nthe benefit. \nPlease Pay Special Attention to These Conditions of \nthe Lost Luggage Benefit: \n• If Your Checked Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, the loss must be reported to the \nCommon Carrier or Cruise Line within the Common Carrier’s or \nCruise Line’s required timeframe for notification to be eligible for \nthis benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the"
2,• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Indemnity Insurance Company of North \nAmerica (“Provider”). Coverage may not be available in all states or \ncertain terms may be different where required by state law. \n• No legal action for a claim may be brought against the Provider \nuntil sixty (60) days after the Provider receives Proof of Loss.,"this benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, it must be reported to the Common \nCarrier or Cruise Line as soon as You exit the Conveyance.\n• This benefit does not apply to any loss caused by or resulting from, \ndirectly or indirectly: War, undeclared war, civil war, insurrection, \nItems You inadvertently leave behind on the Common Carrier are not \nconsidered lost or stolen."


We can see that the reranked results are more relevant to the query.

# Adding Lexical Search

While semantic search is powerful for understanding the meaning behind queries, there are situations where we need more control—such as filtering by specific keywords, brand names, acronyms, or exact phrases. This is where lexical search becomes useful.

Astra DB supports lexical filtering by allowing the use of the $sort clause, which can be combined with vector search to refine results based on exact word matches. This hybrid approach ensures that your application can handle both semantic relevance and precise keyword filtering.

In the following example, we’ll apply lexical filtering to narrow down results to those containing specific terms.

In [51]:
def astra_vector_search_rerank_lexical(query, source, lexical_filter, k=3):
    # Lets recreate the vector store with hybrid search enabled
    vector_store = AstraDBVectorStore(
        collection_name="banking_knowledge_layer",
        api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
        token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
        autodetect_collection=True,
        hybrid_search=HybridSearchMode.ON
    )

    docs_reranked_lexical = vector_store.similarity_search(
        query,
        k=k,
        filter={"source": source},
        lexical_filter=lexical_filter
    )
    
    return docs_reranked_lexical

docs_lexical = astra_vector_search_rerank_lexical(QUERY,SOURCE_DOCUMENT,"police",3)
print_docs(docs_lexical)


Title: Guide to Benefits
Subject: Guide
Page Content: sequence to the loss. War does not include terrorism.
What’s Not Covered?
The Lost Luggage benefit does not apply to loss or theft of 
the following: 
•  Items that have been removed from Carry–On Baggage by You and  
inadvertently left behind on the Common Carrier or Cruise Line
• Documents or valuable papers
• Money
• Securities
• Tickets
• Checks
• Travelers’ checks
• Furs
How Do You File a Claim?
Follow these steps to file Your claim:
--------------------------------------------------------------------------------
Title: Guide to Benefits
Subject: Guide
Page Content: her Account for a Trip, each Covered Person is eligible to receive 
the benefit. 
Please Pay Special Attention to These Conditions of 
the Lost Luggage Benefit: 
•  If Your Checked Baggage is lost, stolen, or damaged by the 
Common Carrier or Cruise Line, the loss must be reported to the 
Common Carrier or Cruise Line within the Common Carrier’s or 
Cruise Line’s re

In [47]:
QUERY = "Is lugagge lost covered?"
SOURCE_DOCUMENT = "docs/BGC10981_SapphireReserve_VisaInfinite.pdf"
LEXICAL_FILTER = "police"

docs = astra_vector_search(QUERY,SOURCE_DOCUMENT,3)
docs_reranked = astra_vector_search_and_rerank(QUERY,SOURCE_DOCUMENT,3)
docs_lexical = astra_vector_search_rerank_lexical(QUERY,SOURCE_DOCUMENT,LEXICAL_FILTER,3)
# Prepare data for the table
data = {
    "Docs": [doc.page_content for doc in docs],
    "Docs Reranked": [doc.page_content for doc in docs_reranked],
    "Docs Lexical": [doc.page_content for doc in docs_lexical]
}

# Create a DataFrame
comparison_df = pd.DataFrame(data)

with pd.option_context('display.max_colwidth', None):
    display(comparison_df)
    
    

/Users/samuel.matioli/work/astra-ibm-demo-2/venv/lib/python3.11/site-packages/langchain_astradb/vectorstores.py:2677: BetaFeatureWarning: Method 'Collection.find_and_rerank' is in beta and might undergo signature or behaviour changes in the future.
  hybrid_reranked_results = self.astra_env.collection.find_and_rerank(


,Docs,Docs Reranked,Docs Lexical
0,"three (3) years after the time for giving proof of loss. Further, no \nlegal action may be brought against the Provider unless all the terms \nof the Guide to Benefits have been complied with fully. \n• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Federal Insurance Company. Coverage \nmay not be available in all states or certain terms may be different \nwhere required by state law.",sequence to the loss. War does not include terrorism.\nWhat’s Not Covered?\nThe Lost Luggage benefit does not apply to loss or theft of \nthe following: \n• Items that have been removed from Carry–On Baggage by You and \ninadvertently left behind on the Common Carrier or Cruise Line\n• Documents or valuable papers\n• Money\n• Securities\n• Tickets\n• Checks\n• Travelers’ checks\n• Furs\nHow Do You File a Claim?\nFollow these steps to file Your claim:,sequence to the loss. War does not include terrorism.\nWhat’s Not Covered?\nThe Lost Luggage benefit does not apply to loss or theft of \nthe following: \n• Items that have been removed from Carry–On Baggage by You and \ninadvertently left behind on the Common Carrier or Cruise Line\n• Documents or valuable papers\n• Money\n• Securities\n• Tickets\n• Checks\n• Travelers’ checks\n• Furs\nHow Do You File a Claim?\nFollow these steps to file Your claim:
1,• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Indemnity Insurance Company of \nNorth America (“Provider”). Coverage may not be available in all \nstates or certain terms may be different where required by state law. \n• No legal action for a claim may be brought against the Provider \nuntil sixty (60) days after the Provider receives Proof of Loss.,"her Account for a Trip, each Covered Person is eligible to receive \nthe benefit. \nPlease Pay Special Attention to These Conditions of \nthe Lost Luggage Benefit: \n• If Your Checked Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, the loss must be reported to the \nCommon Carrier or Cruise Line within the Common Carrier’s or \nCruise Line’s required timeframe for notification to be eligible for \nthis benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the","her Account for a Trip, each Covered Person is eligible to receive \nthe benefit. \nPlease Pay Special Attention to These Conditions of \nthe Lost Luggage Benefit: \n• If Your Checked Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, the loss must be reported to the \nCommon Carrier or Cruise Line within the Common Carrier’s or \nCruise Line’s required timeframe for notification to be eligible for \nthis benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the"
2,• This information is a brief description of the important features \nof this insurance plan. It is not an insurance contract. Insurance \nbenefits are underwritten by Indemnity Insurance Company of North \nAmerica (“Provider”). Coverage may not be available in all states or \ncertain terms may be different where required by state law. \n• No legal action for a claim may be brought against the Provider \nuntil sixty (60) days after the Provider receives Proof of Loss.,"this benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, it must be reported to the Common \nCarrier or Cruise Line as soon as You exit the Conveyance.\n• This benefit does not apply to any loss caused by or resulting from, \ndirectly or indirectly: War, undeclared war, civil war, insurrection, \nItems You inadvertently leave behind on the Common Carrier are not \nconsidered lost or stolen.","this benefit.\n• If Your Carry–On Baggage is lost, stolen, or damaged by the \nCommon Carrier or Cruise Line, it must be reported 

# Recap

In this exercise, you learned:

- How to create an Astra account
- How to create a database
- How to create a collection and use **Vectorize** for embedding generation
- How to retrieve Astra’s access token and endpoint
- How to load data into a collection using Python and LangChain
- How to use vector search to find relevant content for users' questions
- How reranking improves accuracy
- How to combine vector and lexical search with reranking
